## Imports

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import psycopg
from supabase import create_client, Client
from urllib.parse import unquote

## NDMA

In [ ]:
ndma_csv_df = pd.read_csv("../Scrapers/ndma_advisories_bulk.csv")
data_ndma = []

for _, row in ndma_csv_df.iterrows():
    source = row["source_agency"]
    posted_date = row["date"]
    title = row["title"]
    url = row["url"]
    filename, filetype = os.path.splitext(os.path.basename(unquote(row["filename"].split("file=")[1])))
    filetype = filetype.lstrip('.')

    data_ndma.append({
        "source": source,
        "posted_date": posted_date,
        "title": title,
        "url": url,
        "filename": filename,
        "filetype": filetype
    })

ndma_df = pd.DataFrame(data_ndma)
ndma_df.head(1)

## NEOC

In [ ]:
neoc_csv_df = pd.read_csv("../Scrapers/ndma_neoc_projections_bulk.csv")
data_neoc = []

for _, row in neoc_csv_df.iterrows():
    source = row["source_agency"].split("-")[1]
    posted_date = row["date"]
    title = row["title"]
    url = row["url"]
    #filename, filetype = os.path.splitext(row["filename"])
    filename, filetype = os.path.splitext(os.path.basename(unquote(row["filename"].split("?file=")[-1])))
    filetype = filetype.lstrip('.')

    data_neoc.append({
        "source": source,
        "posted_date": posted_date,
        "title": title,
        "url": url,
        "filename": filename,
        "filetype": filetype
    })

neoc_df = pd.DataFrame(data_neoc)
neoc_df.head(1)

## Combination

In [ ]:
documents = pd.concat([ndma_df, neoc_df])
documents['posted_date'] = pd.to_datetime(documents['posted_date'], dayfirst=True)
documents = documents.sort_values(by='posted_date', ascending=False).reset_index(drop=True)
documents['posted_date'] = documents['posted_date'].dt.strftime('%Y-%m-%d')
documents.index += 1

print(len(documents))
print(documents.head())

In [16]:
documents.to_csv("../Scrapers/ndma_neoc_combined_bulk.csv", index_label="id")

## Upload

### Psycopg

#### Connection

In [ ]:
load_dotenv('../local.env', override=True)

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
SUPABASE_DB_URL = os.getenv("SUPABASE_DB_URL")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
conn = psycopg.connect(SUPABASE_DB_URL)
cur = conn.cursor()
print("Connected to Supabase")

#### Upload

In [ ]:
INSERT_QUERY = """
    INSERT INTO documents (source, posted_date, title, url, filename, filetype)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON CONFLICT(id) DO NOTHING
"""

values = [
    (
        row["source"],
        row["posted_date"],
        row["title"],
        row["url"],
        row["filename"],
        row["filetype"],
    )
    for _, row in documents.iterrows()
]

print(f"Inserting {len(values)} records...")
cur.executemany(INSERT_QUERY, values)
conn.commit()

print("Insertion successful")

cur.close()
conn.close()

### Supabase

In [8]:
load_dotenv('../local.env', override=True)

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_SERVICE_KEY  = os.getenv("SUPABASE_SERVICE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_SERVICE_KEY)

In [9]:
doc_records = documents.to_dict('records')

# Upsert data (insert new rows, update existing based on url)
response = supabase.table('documents').upsert(
    doc_records,
    on_conflict='filename'
).execute()

print(f"Upserted {len(response.data)} rows")

Upserted 511 rows
